# Detecció en temps real de mamífers

In [ ]:
import csv
from ultralytics import YOLO
from PIL import Image
import os
import os.path
import shutil
from datetime import datetime

Per coneixer quines espècies el model és capaç de detectar, carreguem el fitxer de classes del model.

In [ ]:
# Obrir el fitxer CSV
with open('models/especies.csv', 'r') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)

    # Initialize an empty list
    especies = []

    # Iterate over each row in the CSV file
    for row in reader:
        # Append the row values to the list
        especies.append(row)

# Print the list of data
for especie in especies:
    print(especie)

Definim la ruta de la carpeta on aniran a parar les imatges provinents de la càmera de fototrampeig.

In [ ]:
#Ruta de la carpeta a vigilar
watchDirectory = r'inbox'

#Ruta de les prediccions
prediction_labels_path = "runs/detect/predict/labels"
prediction_path = "runs/detect/predict"

### Seguidament executarem una serie de funcions d'ajuda.

In [ ]:
#function to return files in a directory
onlyfiles = [f for f in os.listdir(watchDirectory) if os.path.isfile(os.path.join(watchDirectory, f))]

In [ ]:
def readPredictedSpecies():
    detections = []
    label=os.listdir(prediction_labels_path)
    if len(label) != 0: 
        with open('{}/{}'.format(prediction_labels_path,label[0]), 'r') as file:
        # Read classes
            linies = file.readlines()
            for linia in linies:
        # Remove leading/trailing whitespace and newline characters
                especie = linia.split()
                detections.append(especie[0])

        # Print the first value
        return (detections)
    else:
        return

In [ ]:
def storefiles(file):
    especie = readPredictedSpecies()
    if (especie is None) !=0:
        #Si no hi ha cap detecció a la imatge
        try:
            shutil.move('{}/{}'.format(watchDirectory, file), '{}/{}/'.format('predicted', 'nuls'))
        except IOError as io_err:
            os.makedirs(os.path.dirname('{}/{}/'.format('predicted', 'nuls')))
            shutil.move('{}/{}'.format(watchDirectory, file), '{}/{}/'.format('predicted', 'nuls'))
    else:
        try:
            shutil.move('{}/{}'.format(watchDirectory, file), '{}/{}/'.format('predicted', *especies[int(especie[0])]))
        except IOError as io_err:
            os.makedirs(os.path.dirname('{}/{}/'.format('predicted', *especies[int(especie[0])])))
            shutil.move('{}/{}'.format(watchDirectory, file), '{}/{}/'.format('predicted', *especies[int(especie[0])]))
        shutil.copytree('{}/{}'.format(prediction_path, 'labels'), '{}/{}'.format('predicted', *especies[int(especie[0])]), dirs_exist_ok=True)
    shutil.rmtree(prediction_path)

In [ ]:
for file in onlyfiles:
    im = Image.open('{}/{}'.format(watchDirectory, file))
    w,h = im.size
    impalette = im.getpixel((w/2,h/2))
    if impalette[0] == impalette [1] == impalette [2]:
        model = YOLO('models/best_BW.pt')
        imatge = '{}/{}'.format(watchDirectory, file)
        results = model.predict(imatge, save_txt=True, conf=0.6)
    else:
        model = YOLO('models/best_COLOR.pt')
        imatge = '{}/{}'.format(watchDirectory, file)
        results = model.predict(imatge, save_txt=True, conf=0.8)
    readPredictedSpecies
    storefiles(file)